In [2]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import folium
from folium import DivIcon, Marker, Map

In [3]:
df_reviews = pd.read_csv("../data/philadelphia_restaurant_reviews.csv")
df_business = pd.read_json("../data/yelp_academic_dataset_business.json", lines=True)
restaurant_ids = set()
for _, b in df_business.iterrows():
    if b['categories'] and 'Burger' in b['categories'] and 'Restaurants' in b['categories']:
        restaurant_ids.add(b['business_id'])
df_reviews = df_reviews[df_reviews['business_id'].isin(restaurant_ids)]
df = df_reviews.merge(df_business, on='business_id', how='left')
df = df.dropna(subset=['latitude', 'longitude'])
df.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,cvQXRFLCyr0S7EgFb4lZqw,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,5,3,1,1,"On a scale of one to things that are awesome, ...",2009-10-14 01:15:04,Village Whiskey,...,PA,19103,39.951417,-75.173768,4.0,1553,1,"{'RestaurantsReservations': 'False', 'Restaura...","Bars, Nightlife, Whiskey Bars, Burgers, Restau...","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
1,3dVcGYz6GokuEytLrfG8bA,FEI0XkOrUHufSW_rfOTPAA,Dv6RfXLYe1atjgz3Xf4GGw,4,1,0,0,"Good food, reasonably priced, and nice staff. ...",2012-12-04 23:58:36,HipCityVeg,...,PA,19103,39.950774,-75.170397,4.0,1082,1,"{'RestaurantsAttire': 'u'casual'', 'NoiseLevel...","Burgers, Vegetarian, Restaurants, Vegan","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
2,-xM8cH4AeVzgTJBQHlxfUQ,ZaMB7VbOwaARjxdhXjODxA,EtKSTHV5Qx_Q7Aur9o4kQQ,5,0,0,0,Picked up takeout from Village Whiskey (so I c...,2014-11-30 01:55:57,Village Whiskey,...,PA,19103,39.951417,-75.173768,4.0,1553,1,"{'RestaurantsReservations': 'False', 'Restaura...","Bars, Nightlife, Whiskey Bars, Burgers, Restau...","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
3,UqZyX64mgw1vJ1ffnxODjA,cARxOd_5yKCgsCbUZ5ED4Q,EtKSTHV5Qx_Q7Aur9o4kQQ,4,0,0,0,There is a great selection of whiskey/bourbon ...,2014-10-11 12:54:43,Village Whiskey,...,PA,19103,39.951417,-75.173768,4.0,1553,1,"{'RestaurantsReservations': 'False', 'Restaura...","Bars, Nightlife, Whiskey Bars, Burgers, Restau...","{'Wednesday': '16:0-21:0', 'Thursday': '16:0-2..."
4,nCdhMSQA0apDuB_oho5ang,2dyfZNhtyuHdestczTgWjQ,Dv6RfXLYe1atjgz3Xf4GGw,4,0,0,2,Super lunch option\nWatch out for the yoga mat...,2012-08-15 19:17:49,HipCityVeg,...,PA,19103,39.950774,-75.170397,4.0,1082,1,"{'RestaurantsAttire': 'u'casual'', 'NoiseLevel...","Burgers, Vegetarian, Restaurants, Vegan","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."


In [10]:
# TODO:
# 1. Add dots of restaurants to the map
# 2. tweak opacity?
# 3. Add a legend for the colors
# 4. Maybe filter to show grids for x number of review or restaurants?
# 5. Tweak categories to show interesting ones (e.g. burger, pizza, etc.) and NOT 'restaurants' or 'Food'
# 6. Increase grid size?

# Split the datasets into grid cells
df['grid_x'] = (df['longitude'] - df['longitude'].min()) // 0.01
df['grid_y'] = (df['latitude'] - df['latitude'].min()) // 0.01
df['grid'] = df['grid_x'].astype(int).astype(str) + '_' + df['grid_y'].astype(int).astype(str)

restaurant_types = df['categories'].str.split(',').apply(lambda x: [i.strip() for i in x])

# Count the number of reviews in each grid cell
df_counts = df.groupby('grid').size().reset_index(name='count')
# Remove grid cells with less than 50 reviews
# df_counts = df_counts[df_counts['count'] >= 50]
# Add average rating to the grid cells
df_avg_rating = df.groupby('grid')['stars_x'].mean().reset_index(name='avg_rating')
df_counts = df_counts.merge(df_avg_rating, on='grid', how='left')
# Add the 'categories' column to the grid cells
df_categories = df[['grid', 'categories']].drop_duplicates(subset='grid')
df_counts = df_counts.merge(df_categories, on='grid', how='left')
# Create a grid map of the city
grid_map = folium.Map(
                      max_bounds=True, 
                      location=[df['latitude'].mean(), 
                                df['longitude'].mean()], 
                      zoom_start=12,
                      min_zoom=11,
                      min_lat=df['latitude'].min() - 0.1,
                      max_lat=df['latitude'].max() + 0.1,
                      min_lon=df['longitude'].min() - 0.1,
                      max_lon=df['longitude'].max() + 0.1,
)
# Add the grid cells to the map
cmap = plt.get_cmap('RdYlGn')  # This returns a colormap that can be mapped from 0 to 1

show = True
# iterate over the top 10 restaurant types
for restaurant_type in restaurant_types.explode().value_counts().head(10).index:
    fg = folium.FeatureGroup(name=restaurant_type, show=show)
    show = False  # Only show the first feature group
    subset = df_counts[df_counts['categories'].str.contains(restaurant_type)]
    
    
    for _, row in subset.iterrows():
        grid_x, grid_y = map(int, row['grid'].split('_'))
        lat = df['latitude'].min() + (grid_y + 0.5) * 0.01
        lon = df['longitude'].min() + (grid_x + 0.5) * 0.01
        
        # Normalize rating to [0,1] and convert to hex
        normalized_rating = row['avg_rating'] / 5
        rgba = cmap(normalized_rating)
        hex_color = matplotlib.colors.rgb2hex(rgba)
        
        folium.Rectangle(
            bounds=[[lat - 0.005, lon - 0.005], [lat + 0.005, lon + 0.005]],
            color=None,
            fill=True,
            fill_color=hex_color,
            fill_opacity=0.7,
            popup=f'Count: {row["count"]}<br>Avg Rating: {row["avg_rating"]}',
        ).add_to(fg)
        
        # Add a marker for the restaurants in the grid cell
        restaurants_in_cell = df[(df['grid'] == row['grid']) & (df['categories'].str.contains(restaurant_type))]
        for _, restaurant in restaurants_in_cell.iterrows():
            folium.Marker(
                location=[restaurant['latitude'], restaurant['longitude']],
                icon=DivIcon(
                    html=f'<div style="font-size: 10pt; color: black;">{restaurant["name"]}</div>',
                    class_name='marker',
                ),
                popup=f'{restaurant["name"]}<br>Rating: {restaurant["stars_x"]}',
            ).add_to(fg)
        
    fg.add_to(grid_map)
    
# Add layer control
folium.LayerControl(collapsed=False).add_to(grid_map)

grid_map

C:\Users\soren\AppData\Local\Temp\ipykernel_5876\2420164599.py:46: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset = df_counts[df_counts['categories'].str.contains(restaurant_type)]


KeyboardInterrupt: 

In [5]:
# get the average rating for each grid cell
df_avg = df.groupby('grid')['stars_x'].mean().reset_index(name='avg_stars')


In [6]:
# Find unique values in postal_code column
unique_postal_codes = df['postal_code'].unique()
print("Unique postal codes:", unique_postal_codes)
# Count the number of reviews for each postal code
postal_code_counts = df['postal_code'].value_counts()
print("Number of reviews for each postal code:")
print(postal_code_counts)

Unique postal codes: ['19103' '19130' '19107' '19102' '19146' '19152' '19104' '19147' '19114'
 '19122' '19134' '19129' '19145' '19106' '19121' '19133' '19132' '19154'
 '19124' '19143' '19136' '19140' '19123' '19125' '19135' '19127' '19137'
 '19148' '19128' '19153' '19111' '19087' '19151' '19116' '19126' '19149'
 '19150' '19144' '19141' '19139' '19115' '19120' '19131' '19118' '19119'
 '19142']
Number of reviews for each postal code:
postal_code
19103    6055
19107    4211
19106    4019
19102    3869
19104    3495
19147    2920
19123    2900
19125    1764
19146    1585
19148    1374
19127     906
19145     835
19152     722
19130     715
19128     654
19114     544
19129     524
19131     497
19151     454
19153     444
19143     378
19134     265
19149     263
19118     206
19136     206
19154     195
19122     188
19135     182
19121     173
19140     147
19115     132
19124     128
19111     127
19137      95
19141      86
19116      82
19132      77
19139      65
19150      62
19144 

In [7]:
# Create map of postal codes to average ratings and mean longitude/latitude
postal_code_avg = df.groupby('postal_code').agg({'stars_x': 'mean', 'latitude': 'mean', 'longitude': 'mean'}).reset_index()
postal_code_avg.columns = ['postal_code', 'avg_rating', 'latitude', 'longitude']
postal_code_avg['avg_rating'] = postal_code_avg['avg_rating'].round(2)
# add a column for the number of reviews in each postal code
postal_code_avg['review_count'] = df.groupby('postal_code')['stars_x'].count().values
postal_code_avg.head()

,postal_code,avg_rating,latitude,longitude,review_count
0,19087,3.73,40.041472,-75.363647,15
1,19102,3.33,39.950633,-75.164726,3869
2,19103,3.80,39.951203,-75.170873,6055
3,19104,3.49,39.954443,-75.196575,3495
4,19106,3.84,39.949996,-75.145577,4019


In [8]:
# print range of review counts
print("Range of review counts:", postal_code_avg['review_count'].min(), "-", postal_code_avg['review_count'].max())
print("Average review count:", postal_code_avg['review_count'].mean())

Range of review counts: 7 - 6055
Average review count: 907.5217391304348


In [9]:

center = [39.952583, -75.165222]  # Center of Philadelphia

# Create a folium map
m = folium.Map(location=center, zoom_start=12)
# Add postal code markers to the map
for _, row in postal_code_avg.iterrows():
	folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=DivIcon(
            icon_size=(250,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size: 12pt; color : {'red' if row['avg_rating'] < 3 else 'green'}">{row['postal_code']}<br>Avg Rating: {row['avg_rating']}<br>Review Count: {row['review_count']}</div>""",
            )
    ).add_to(m)

# Show the map
m